In [ ]:
from os import chdir
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
chdir("/content/drive/MyDrive/Eliott/files/")

%pip install ipython-autotime

Mounted at /content/drive
     |████████████████████████████████| 308 kB 7.6 MB/s 
     |████████████████████████████████| 210 kB 50.3 MB/s 
     |████████████████████████████████| 81 kB 9.8 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 146 kB 53.1 MB/s 
     |████████████████████████████████| 113 kB 54.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=7f377fe7937ecd9c287b6d82c5fdf95b4f52eff9f2e3a49466ad702632597521
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


#### CHANTRE Honorine  CHAH2807
#### THOMAS Eliott THOE2303

In [ ]:
# IMPORTS

import preprocessing as pp
import hyperparameters as hp
import RNN as rnn
import optimising as opt
import json
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier

%load_ext autoreload
%autoreload 2

%load_ext autotime


time: 654 µs (started: 2022-04-15 16:22:19 +00:00)


In [ ]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=None, original_class=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=None, original_class=False)

time: 16.3 s (started: 2022-04-15 16:22:19 +00:00)


In [ ]:
X_train[0]

'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order'

time: 4.1 ms (started: 2022-04-15 16:22:35 +00:00)


In [ ]:
pipeline_sgd = hp.pipeline_sgd
pipeline_gb = hp.pipeline_gb
pipeline_rf = hp.pipeline_rf
pipeline_lr = hp.pipeline_lr
pipeline_per = hp.pipeline_per
pipeline_svc = hp.pipeline_svc

time: 2.33 ms (started: 2022-04-15 16:22:35 +00:00)


# Hyperparameters Tuning

### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [ ]:
with open('data.json') as json_file:
    dico = json.load(json_file)

time: 264 ms (started: 2022-04-15 23:58:24 +00:00)


### SGD

In [ ]:
%%time
grid_sgd = hp.grid_search("SGD", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Score:  0.8668915077832064
Best Params:  {'clf__alpha': 1e-05, 'clf__penalty': 'l1', 'tfidf__max_df': 0.3, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 14.4 s, sys: 2.6 s, total: 17 s
Wall time: 8min 30s
time: 8min 30s (started: 2022-04-15 16:22:47 +00:00)


In [ ]:
hp.if_save("SGD", dico, grid_sgd)

#### No new best score for  SGD
time: 2.05 ms (started: 2022-04-15 16:31:18 +00:00)


### Gradient boosting

In [ ]:
%%time
grid_gb = hp.grid_search("GB", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Score:  0.581
Best Params:  {'clf__learning_rate': 0.2, 'clf__n_estimators': 200, 'tfidf__max_df': 0.3, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
CPU times: user 7.96 s, sys: 490 ms, total: 8.45 s
Wall time: 9min 54s
time: 9min 54s (started: 2022-04-15 16:31:18 +00:00)


In [ ]:
hp.if_save("GB", dico, grid_gb)

#### No new best score for  GB
time: 1.63 ms (started: 2022-04-15 16:41:12 +00:00)


### Random Forest

In [ ]:
%%time
grid_rf = hp.grid_search("RF", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Score:  0.594
Best Params:  {'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'tfidf__max_df': 0.6, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
CPU times: user 8.42 s, sys: 369 ms, total: 8.79 s
Wall time: 2min 35s
time: 2min 35s (started: 2022-04-15 18:26:41 +00:00)


In [ ]:
hp.if_save('RF', dico, grid_rf)

#### No new best score for  RF
time: 1.59 ms (started: 2022-04-15 18:29:16 +00:00)


### Logistic Regression

In [ ]:
%%time
grid_lr = hp.grid_search("LR", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Score:  0.6140000000000001
Best Params:  {'clf__C': 10, 'clf__tol': 1e-09, 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 15.7 s, sys: 2.83 s, total: 18.5 s
Wall time: 4min 41s
time: 4min 41s (started: 2022-04-15 22:24:12 +00:00)


In [ ]:
hp.if_save('LR', dico, grid_lr)

#### No new best score for  LR
time: 2.95 ms (started: 2022-04-15 22:28:54 +00:00)


### Perceptron

In [ ]:
%%time
grid_per = hp.grid_search("PER", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best Score:  0.8097043608680832
Best Params:  {'clf__alpha': 5.05e-06, 'clf__penalty': 'l1', 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 3s, sys: 8.39 s, total: 1min 11s
Wall time: 37min 26s
time: 37min 26s (started: 2022-04-15 22:45:14 +00:00)


In [ ]:
hp.if_save('PER', dico, grid_per)

#### No new best score for  PER
time: 2.77 ms (started: 2022-04-15 23:58:39 +00:00)


### LinearSVC

In [ ]:
%%time
grid_svc = hp.grid_search("SVC", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
720 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-package

Best Score:  0.8602693724198008
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1', 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 1s, sys: 6.82 s, total: 1min 8s
Wall time: 44min 48s
time: 44min 48s (started: 2022-04-15 23:58:54 +00:00)


In [ ]:
hp.if_save('SVC', dico, grid_svc)

#### No new best score for  SVC
time: 2.54 ms (started: 2022-04-16 00:43:43 +00:00)


### Charging the best parameters for training

In [ ]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)

data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['PER']
data_svc = data['SVC']

time: 5.19 ms (started: 2022-04-16 00:50:42 +00:00)


### Training on all training data and testing on test data

In [ ]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train, predictions_train, target_names=['Negative','Neutral' ,'Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test,predictions_test, target_names=['Negative', 'Neutral', 'Positive']))
    print('------------------------------------------------------\n')


time: 5.6 ms (started: 2022-04-16 00:50:46 +00:00)


### SGD

In [ ]:
data_sgd

{'best_score': 0.8769590395125022,
 'params': {'clf__alpha': 1.2000000000000002e-05,
  'clf__penalty': 'l1',
  'tfidf__max_df': 0.75,
  'tfidf__min_df': 1,
  'tfidf__ngram_range': [1, 1]}}

time: 3.25 ms (started: 2022-04-16 00:50:48 +00:00)


In [ ]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**(data_sgd["params"]),clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 10.1 ms (started: 2022-04-16 00:50:48 +00:00)


In [ ]:
%%time

model_sgd_best.fit(X_train,y_train)


CPU times: user 2.17 s, sys: 213 ms, total: 2.38 s
Wall time: 1.58 s


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 1.59 s (started: 2022-04-16 00:50:49 +00:00)


In [ ]:
testing(model_sgd_best)

train_accuracy : 0.9310947539450039 
test_accuracy : 0.8700370566437269  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.93      0.93      0.93     15317
     Neutral       0.92      0.89      0.90      7392
    Positive       0.93      0.95      0.94     17912

    accuracy                           0.93     40621
   macro avg       0.93      0.92      0.93     40621
weighted avg       0.93      0.93      0.93     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.89      0.87      0.88      1631
     Neutral       0.83      0.83      0.83       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.87      3778
   macro avg       0.86      0.86      0.86      3778
weighted avg       0.87      0.87      0.

### Random Forest

In [ ]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**(data_rf["params"]),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 7.98 ms (started: 2022-04-16 00:50:52 +00:00)


In [ ]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 26 s, sys: 20.6 ms, total: 26.1 s
Wall time: 7.58 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 7.6 s (started: 2022-04-16 00:50:52 +00:00)


In [ ]:
testing(model_rf_best)

train_accuracy : 0.8667930380837497 
test_accuracy : 0.6974589730015881  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.90      0.90      0.90     15317
     Neutral       0.90      0.58      0.71      7392
    Positive       0.83      0.96      0.89     17912

    accuracy                           0.87     40621
   macro avg       0.88      0.81      0.83     40621
weighted avg       0.87      0.87      0.86     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.77      0.67      0.72      1631
     Neutral       0.73      0.40      0.52       604
    Positive       0.64      0.84      0.73      1543

    accuracy                           0.70      3778
   macro avg       0.71      0.64      0.65      3778
weighted avg       0.71      0.70      0.

### Gradient Boosting 

In [ ]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**(data_gb["params"]))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=200))])

time: 7.49 ms (started: 2022-04-16 00:51:02 +00:00)


In [ ]:
%%time
model_gb_best.fit(X_train,y_train)

CPU times: user 3min 57s, sys: 28.7 ms, total: 3min 57s
Wall time: 3min 56s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=200))])

time: 3min 56s (started: 2022-04-16 00:51:02 +00:00)


In [ ]:
testing(model_gb_best)

train_accuracy : 0.7982816769651165 
test_accuracy : 0.7506617257808365  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.85      0.79      0.82     15317
     Neutral       0.75      0.65      0.70      7392
    Positive       0.78      0.87      0.82     17912

    accuracy                           0.80     40621
   macro avg       0.79      0.77      0.78     40621
weighted avg       0.80      0.80      0.80     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.83      0.72      0.77      1631
     Neutral       0.67      0.62      0.64       604
    Positive       0.71      0.84      0.77      1543

    accuracy                           0.75      3778
   macro avg       0.74      0.72      0.73      3778
weighted avg       0.76      0.75      0.

### Logistic Regression

In [ ]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**(data_lr['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 7.56 ms (started: 2022-04-16 00:55:00 +00:00)


In [ ]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 1.51 s, sys: 65.2 ms, total: 1.58 s
Wall time: 23.7 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 23.7 s (started: 2022-04-16 00:55:00 +00:00)


In [ ]:
testing(model_lr_best)

train_accuracy : 0.9775239408187884 
test_accuracy : 0.8213340391741663  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.97      0.98      0.98     15317
     Neutral       0.99      0.94      0.97      7392
    Positive       0.98      0.99      0.98     17912

    accuracy                           0.98     40621
   macro avg       0.98      0.97      0.98     40621
weighted avg       0.98      0.98      0.98     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.83      1631
     Neutral       0.72      0.72      0.72       604
    Positive       0.83      0.86      0.85      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.80      0.80      3778
weighted avg       0.82      0.82      0.

### Perceptron

In [ ]:
model_per_best = pipeline_per
model_per_best.set_params(**(data_per['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 8.08 ms (started: 2022-04-16 00:55:25 +00:00)


In [ ]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.73 s, sys: 290 ms, total: 2.02 s
Wall time: 1.41 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 1.42 s (started: 2022-04-16 00:55:25 +00:00)


In [ ]:
testing(model_per_best)

train_accuracy : 0.904089017995618 
test_accuracy : 0.8194812069878242  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.91      0.91      0.91     15317
     Neutral       0.88      0.80      0.84      7392
    Positive       0.91      0.94      0.92     17912

    accuracy                           0.90     40621
   macro avg       0.90      0.88      0.89     40621
weighted avg       0.90      0.90      0.90     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.84      1631
     Neutral       0.72      0.69      0.70       604
    Positive       0.82      0.87      0.84      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.79      0.80      3778
weighted avg       0.82      0.82      0.8

### SVC

In [ ]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**(data_svc['params']))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 6.69 ms (started: 2022-04-16 00:55:28 +00:00)


In [ ]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 8.1 s, sys: 21.7 ms, total: 8.12 s
Wall time: 8.06 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 8.07 s (started: 2022-04-16 00:55:28 +00:00)


In [ ]:
testing(model_svc_best)

train_accuracy : 0.9460623815267964 
test_accuracy : 0.8642138697723664  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.95      0.95     15317
     Neutral       0.95      0.90      0.93      7392
    Positive       0.95      0.96      0.95     17912

    accuracy                           0.95     40621
   macro avg       0.95      0.94      0.94     40621
weighted avg       0.95      0.95      0.95     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.86      0.87      1631
     Neutral       0.81      0.80      0.80       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.86      0.

### Voting Classifier

In [ ]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


time: 5.32 ms (started: 2022-04-16 00:55:37 +00:00)


In [ ]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 4min 13s, sys: 1.02 s, total: 4min 14s
Wall time: 4min 39s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                               

time: 4min 39s (started: 2022-04-16 00:55:37 +00:00)


In [ ]:
testing(eclf1)

train_accuracy : 0.9531030747642845 
test_accuracy : 0.86500794070937  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.97      0.95     15317
     Neutral       0.97      0.90      0.93      7392
    Positive       0.96      0.96      0.96     17912

    accuracy                           0.95     40621
   macro avg       0.96      0.94      0.95     40621
weighted avg       0.95      0.95      0.95     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.87      0.88      0.87      1631
     Neutral       0.84      0.78      0.81       604
    Positive       0.87      0.88      0.88      1543

    accuracy                           0.87      3778
   macro avg       0.86      0.85      0.85      3778
weighted avg       0.86      0.87      0.86

In [ ]:
%%time

eclf2.fit(X_train,y_train)

CPU times: user 13 s, sys: 617 ms, total: 13.6 s
Wall time: 33.5 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_df=0.75,
                                                               ngram_range=[1,
                                                                            1])),
                                              ('clf',
                                               SGDClassifier(alpha=1.2000000000000002e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                      

time: 33.6 s (started: 2022-04-16 01:00:25 +00:00)


In [ ]:
testing(eclf2)

train_accuracy : 0.9552448241057581 
test_accuracy : 0.8642138697723664  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.97      0.96     15317
     Neutral       0.96      0.92      0.94      7392
    Positive       0.96      0.96      0.96     17912

    accuracy                           0.96     40621
   macro avg       0.96      0.95      0.95     40621
weighted avg       0.96      0.96      0.96     40621

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.87      0.88      0.87      1631
     Neutral       0.82      0.79      0.81       604
    Positive       0.88      0.88      0.88      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.86      0.

# Reccurent Network

In [ ]:
optim_best_params = opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=30, CV=5)

[I 2022-04-16 01:11:11,002] A new study created in memory with name: no-name-256692eb-42f1-4570-ab55-a9d89185e77c


{'epochs': 4, 'batch_size': 18, 'embedding_dim': 62, 'units': 159, 'dropout': 0.5, 'n_neurons': 66}
Epoch 1/4
2031/2031 [==============================] - 34s 15ms/step - loss: 0.1256 - accuracy: 0.7557 - val_loss: 0.0717 - val_accuracy: 0.8639
Epoch 2/4
2031/2031 [==============================] - 30s 15ms/step - loss: 0.0662 - accuracy: 0.8959 - val_loss: 0.0637 - val_accuracy: 0.8789
Epoch 3/4
2031/2031 [==============================] - 30s 15ms/step - loss: 0.0468 - accuracy: 0.9385 - val_loss: 0.0638 - val_accuracy: 0.8769
Epoch 4/4
2031/2031 [==============================] - 30s 15ms/step - loss: 0.0369 - accuracy: 0.9567 - val_loss: 0.0623 - val_accuracy: 0.8789
Epoch 1/4
2031/2031 [==============================] - 30s 15ms/step - loss: 0.0322 - accuracy: 0.9644 - val_loss: 0.0620 - val_accuracy: 0.8794
Epoch 2/4
2031/2031 [==============================] - 30s 15ms/step - loss: 0.0290 - accuracy: 0.9697 - val_loss: 0.0669 - val_accuracy: 0.8686
Epoch 3/4
2031/2031 [=========

[I 2022-04-16 01:21:25,918] Trial 0 finished with value: 0.8706374526023865 and parameters: {'epochs': 4, 'batch_size': 18, 'embedding_dim': 62, 'units': 159, 'dropout': 0.5, 'n_neurons': 66}. Best is trial 0 with value: 0.8706374526023865.


{'epochs': 3, 'batch_size': 55, 'embedding_dim': 35, 'units': 271, 'dropout': 0.30000000000000004, 'n_neurons': 41}
Epoch 1/3
665/665 [==============================] - 20s 23ms/step - loss: 0.1357 - accuracy: 0.7200 - val_loss: 0.0696 - val_accuracy: 0.8728
Epoch 2/3
665/665 [==============================] - 14s 21ms/step - loss: 0.0669 - accuracy: 0.8978 - val_loss: 0.0641 - val_accuracy: 0.8809
Epoch 3/3
665/665 [==============================] - 14s 22ms/step - loss: 0.0465 - accuracy: 0.9394 - val_loss: 0.0618 - val_accuracy: 0.8806
Epoch 1/3
665/665 [==============================] - 14s 22ms/step - loss: 0.0366 - accuracy: 0.9567 - val_loss: 0.0647 - val_accuracy: 0.8764
Epoch 2/3
665/665 [==============================] - 14s 22ms/step - loss: 0.0309 - accuracy: 0.9661 - val_loss: 0.0614 - val_accuracy: 0.8853
Epoch 3/3
665/665 [==============================] - 14s 22ms/step - loss: 0.0279 - accuracy: 0.9697 - val_loss: 0.0614 - val_accuracy: 0.8801
Epoch 1/3
665/665 [=======

[I 2022-04-16 01:25:09,626] Trial 1 finished with value: 0.8718680739402771 and parameters: {'epochs': 3, 'batch_size': 55, 'embedding_dim': 35, 'units': 271, 'dropout': 0.30000000000000004, 'n_neurons': 41}. Best is trial 1 with value: 0.8718680739402771.


...

{'epochs': 3, 'batch_size': 19, 'embedding_dim': 8, 'units': 404, 'dropout': 0.2, 'n_neurons': 94}
Epoch 1/3
1925/1925 [==============================] - 42s 19ms/step - loss: 0.1287 - accuracy: 0.7271 - val_loss: 0.0737 - val_accuracy: 0.8681
Epoch 2/3
1925/1925 [==============================] - 36s 19ms/step - loss: 0.0613 - accuracy: 0.8973 - val_loss: 0.0606 - val_accuracy: 0.8838
Epoch 3/3
1925/1925 [==============================] - 36s 19ms/step - loss: 0.0415 - accuracy: 0.9371 - val_loss: 0.0610 - val_accuracy: 0.8794
Epoch 1/3
1925/1925 [==============================] - 36s 19ms/step - loss: 0.0322 - accuracy: 0.9536 - val_loss: 0.0640 - val_accuracy: 0.8740
Epoch 2/3
1925/1925 [==============================] - 36s 19ms/step - loss: 0.0269 - accuracy: 0.9621 - val_loss: 0.0618 - val_accuracy: 0.8816
Epoch 3/3
1925/1925 [==============================] - 36s 19ms/step - loss: 0.0237 - accuracy: 0.9669 - val_loss: 0.0649 - val_accuracy: 0.8742
Epoch 1/3
1925/1925 [==========

[I 2022-04-16 04:11:04,935] Trial 28 finished with value: 0.871868085861206 and parameters: {'epochs': 3, 'batch_size': 19, 'embedding_dim': 8, 'units': 404, 'dropout': 0.2, 'n_neurons': 94}. Best is trial 10 with value: 0.8840265870094299.


{'epochs': 2, 'batch_size': 18, 'embedding_dim': 58, 'units': 471, 'dropout': 0.5, 'n_neurons': 112}
Epoch 1/2
2031/2031 [==============================] - 61s 28ms/step - loss: 0.1216 - accuracy: 0.7620 - val_loss: 0.0743 - val_accuracy: 0.8634
Epoch 2/2
2031/2031 [==============================] - 57s 28ms/step - loss: 0.0635 - accuracy: 0.8985 - val_loss: 0.0628 - val_accuracy: 0.8772
Epoch 1/2
2031/2031 [==============================] - 56s 27ms/step - loss: 0.0449 - accuracy: 0.9382 - val_loss: 0.0585 - val_accuracy: 0.8895
Epoch 2/2
2031/2031 [==============================] - 56s 27ms/step - loss: 0.0345 - accuracy: 0.9576 - val_loss: 0.0628 - val_accuracy: 0.8760
Epoch 1/2
2031/2031 [==============================] - 56s 27ms/step - loss: 0.0294 - accuracy: 0.9665 - val_loss: 0.0615 - val_accuracy: 0.8799
Epoch 2/2
2031/2031 [==============================] - 56s 28ms/step - loss: 0.0265 - accuracy: 0.9702 - val_loss: 0.0605 - val_accuracy: 0.8841
Epoch 1/2
2031/2031 [========

[I 2022-04-16 04:21:30,706] Trial 29 finished with value: 0.8763475298881531 and parameters: {'epochs': 2, 'batch_size': 18, 'embedding_dim': 58, 'units': 471, 'dropout': 0.5, 'n_neurons': 112}. Best is trial 10 with value: 0.8840265870094299.


Number of finished trials:  30
Best value: 0.8840265870094299
Best params: {'epochs': 2, 'batch_size': 27, 'embedding_dim': 50, 'units': 393, 'dropout': 0.1, 'n_neurons': 122}
time: 3h 10min 19s (started: 2022-04-16 01:11:11 +00:00)


In [ ]:
rnn_model = rnn.RNN(X_train, y_train, y_test, embedding_dim=optim_best_params['embedding_dim'], units=optim_best_params['units'], dropout=optim_best_params['dropout'], n_neurons=optim_best_params['n_neurons'])

##### CREATING RNN #####
Vocabulary size: 41230

Example:

Sentence:
Was at the supermarket today Didnt buy toilet paper

After tokenizing :
[68, 12, 1, 18, 115, 530, 114, 100, 107]

After padding :
[ 68  12   1  18 115 530 114 100 107   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 50)            2061500   
                                                                 
 bidirectional (Bidirectiona  (None, 63, 786)          1395936   
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 786)              0      

In [ ]:
rnn_model.train(epochs=optim_best_params['epochs'], batch_size=optim_best_params['batch_size']) # Train the model

In [ ]:
print(rnn_model.history.history['val_accuracy'][-1])

0.8899827599525452
time: 1.78 ms (started: 2022-04-16 04:22:48 +00:00)


In [ ]:
rnn_model.print_history() 

##### PRINTING HISTORY #####


##### PRINTING HISTORY COMPLETED #####
time: 128 ms (started: 2022-04-16 04:22:48 +00:00)


In [76]:
rnn_model.test(X_test) # Test the model

##### TESTING #####
119/119 [==============================] - 1s 8ms/step - loss: 0.0627 - accuracy: 0.8759
Test loss: 0.06265214830636978
Test Accuracy: 0.8758602142333984
##### TESTING COMPLETED #####


(0.06265214830636978, 0.8758602142333984)

time: 1.42 s (started: 2022-04-16 05:17:27 +00:00)


### Personal tests

In [77]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

time: 1.61 ms (started: 2022-04-16 05:17:34 +00:00)


In [78]:
predictions_out = eclf2.predict(tests_perso)


time: 9.79 ms (started: 2022-04-16 05:17:34 +00:00)


In [79]:
predictions_out

array([-1,  1,  1])

time: 3.27 ms (started: 2022-04-16 05:17:34 +00:00)


One can see that the first and second sentences are correctly classified as negative and positive. However the third sentence is wrongly classified as positive. It's because the tfidf vectorizer is just counting the number of occurences of each word in the text. It doesn't take into account the importance of the word. As a result even though the third sentence is negative, it's classified as positive because it contains the word "love" and "birthday"and "party".